In [1]:
import sys
sys.path.append("/home/jerryin/jupyter_proj/csgo/")


from Util.util import Constant, getProxy
from Util import util
import time
import requests
import pickle
from Ip import ip
from Account.rawAccountPool import raw_accounts
from selenium import webdriver
from selenium.webdriver.common.proxy import Proxy, ProxyType
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import re
from Sms.sms import sms
'''

2种账号：
    Buyer（购买者）账号和Viewer(浏览者), Viewer获取数据并记录可购买的商品，Buyer购买商品。

1. Account -> 可用于Buyer和Viewer 这两个类继承Account类
    Buyer和Viewer获取、更新ip方式相同
    Buyer和Viewer登录方式相同
    Buyer和Viewer获取账号方式不同
    Viewer直接从接口获取手机号，并通过手机号验证登录后，获取10天的免登录状态，用这10天cookies获取数据，若过期则直接换下一个手机号。
    Buyer通过excel或json获取账号和密码，通过密码登录，获取10天的免登录状态，用这10天的cookies购买商品，若过期则重新登录。
    登录完成后把账号存放于硬盘中。
2. AccountPool -> 一组Account的List，存放Account。
    从硬盘中读取一个个Account信息，并验证可用性。
    若账号不可用，则直接删除，若IP不可用，则用Account来更新IP。

'''

In [ ]:
class Account(object):
    '''
    methods:

    '''
    def __init__(self) -> None:
        self.ip = self.getIp()
        self.phone = '' 
        # self.login_time = 0
        self.cookie = ''
        self.csrf_token = '' # the token for visiting website
        self.status = -1
    
    def isIpAlive(self) -> bool:
        '''
            check the ip banded with the account is alive or not.
        '''
        return ip.checkIpAlive(self.ip)
    
    def accountStatus(self) -> int:
        '''
        return: 
            0: login and market available
            1: did not login
            -1: can not access market or other problems  
        '''
        headers = {
            ':method': 'GET',
            ':path': Constant.account_availability_check_re_addr,
            'cookie': self.cookie + 'csrf_token=' + self.csrf_token,
            'sec-fetch-user': '?1',
            'upgrade-insecure-requests': '1',
        }
        Constant.sessions.proxies = getProxy(self.ip)
        resp = Constant.sessions.get(Constant.account_availability_check_ab_addr, headers = headers)
        code = resp.json()['code']
        print('check account status, the code is: ' + code + '\n')
        if code == 'OK' or code == 'ok':
            return 0
        elif code == 'Login Required':
            return 1
        else:
            return -1